# Importing libraries

In [543]:
from pyzotero import zotero
import os
import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px
import pycountry
import re
import pandas as pd

In [544]:
"# All items in the Zotero Intelligence bibliography library".index("All")

2

In [545]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [546]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

for item in items:
    # Extracting various types of creators
    authors = ""
    if 'creators' in item['data']:
        authors_info = item['data']['creators']
        author_names = []
        for author in authors_info:
            creator_type = author.get('creatorType')
            valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
            
            if item['data']['itemType'] == 'bookSection' and creator_type == 'editor':
                continue  # Skip adding 'editor' for Book chapters
            
            if creator_type in valid_types:
                if 'firstName' in author and 'lastName' in author:
                    author_names.append(author['firstName'] + ' ' + author['lastName'])
                elif 'name' in author:
                    author_names.append(author['name'])
                
        authors = ', '.join(author_names)
        
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        authors,  # Add the concatenated authors string
        item['data'].get('publisher'),
        item['data'].get('publicationTitle'),
        item['data']['dateAdded'],
        item['data']['collections']
        )) 
# pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

mapping_types = {
    'thesis': 'Thesis',
    'journalArticle': 'Journal article',
    'book': 'Book',
    'bookSection': 'Book chapter',
    'blogPost': 'Blog post',
    'videoRecording': 'Video',
    'podcast': 'Podcast',
    'magazineArticle': 'Magazine article',
    'webpage': 'Webpage',
    'newspaperArticle': 'Newspaper article',
    'report': 'Report',
    'forumPost': 'Forum post',
    'manuscript': 'Manuscript',
    'document': 'Document',
    'conferencePaper': 'Conference paper',
    'film': 'Film',
    'presentation': 'Presentation',
    'audioRecording':'Podcast'
}
df['Publication type'] = df['Publication type'].replace(mapping_types)

mapping_publisher = {
    'Taylor & Francis Group': 'Taylor and Francis',
    'Taylor and Francis': 'Taylor and Francis',
    'Taylor & Francis': 'Taylor and Francis',
    'Routledge': 'Routledge',
    'Routledge Handbooks Online': 'Routledge',
    'Praeger Security International': 'Praeger',
    'Praeger': 'Praeger'
}
df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

mapping_journal = {
    'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'Intelligence and national security': 'Intelligence and National Security',
    'Intelligence and National Security': 'Intelligence and National Security',
    'Intelligence & National Security': 'Intelligence and National Security'
}

df['Journal'] = df['Journal'].replace(mapping_journal)

# df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [547]:
# items = zot.everything(zot.top())

# data3=[]
# columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

# for item in items:
#     # Extracting various types of creators
#     authors = ""
#     if 'creators' in item['data']:
#         authors_info = item['data']['creators']
#         author_names = []
#         for author in authors_info:
#             creator_type = author.get('creatorType')
#             valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
#             if creator_type in valid_types:  # Include all identified creator types
#                 if 'firstName' in author and 'lastName' in author:
#                     author_names.append(author['firstName'] + ' ' + author['lastName'])
#                 elif 'name' in author:
#                     author_names.append(author['name'])  # Handling cases where 'firstName' and 'lastName' are absent
                
#         authors = ', '.join(author_names)
        
#     data3.append((
#         item['data']['title'], 
#         item['data']['itemType'], 
#         item['data']['url'], 
#         item['data']['abstractNote'], 
#         item['links']['alternate']['href'],
#         item['data'].get('date'),
#         authors,  # Add the concatenated authors string
#         item['data'].get('publisher'),
#         item['data'].get('publicationTitle'),
#         item['data']['dateAdded'],
#         item['data']['collections']
#         )) 
# # pd.set_option('display.max_colwidth', None)

# df = pd.DataFrame(data3, columns=columns3)

# mapping_types = {
#     'thesis': 'Thesis',
#     'journalArticle': 'Journal article',
#     'book': 'Book',
#     'bookSection': 'Book chapter',
#     'blogPost': 'Blog post',
#     'videoRecording': 'Video',
#     'podcast': 'Podcast',
#     'magazineArticle': 'Magazine article',
#     'webpage': 'Webpage',
#     'newspaperArticle': 'Newspaper article',
#     'report': 'Report',
#     'forumPost': 'Forum post',
#     'manuscript': 'Manuscript',
#     'document': 'Document',
#     'conferencePaper': 'Conference paper',
#     'film': 'Film',
#     'presentation': 'Presentation',
#     'audioRecording':'Podcast'
# }
# df['Publication type'] = df['Publication type'].replace(mapping_types)

# mapping_publisher = {
#     'Taylor & Francis Group': 'Taylor and Francis',
#     'Taylor and Francis': 'Taylor and Francis',
#     'Taylor & Francis': 'Taylor and Francis',
#     'Routledge': 'Routledge',
#     'Routledge Handbooks Online': 'Routledge',
#     'Praeger Security International': 'Praeger',
#     'Praeger': 'Praeger'
# }
# df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# # df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# # df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# # df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

# mapping_journal = {
#     'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'Intelligence and national security': 'Intelligence and National Security',
#     'Intelligence and National Security': 'Intelligence and National Security',
#     'Intelligence & National Security': 'Intelligence and National Security'
# }

# df['Journal'] = df['Journal'].replace(mapping_journal)

# # df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# # df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [548]:
def zotero_collections2(library_id, library_type):
    collections = zot.collections()
    data = [(item['data']['key'], item['data']['name'], item['meta']['numItems'], item['links']['alternate']['href']) for item in collections]
    df_collections = pd.DataFrame(data, columns=['Key', 'Name', 'Number', 'Link'])
    return df_collections
df_collections_2 = zotero_collections2(library_id, library_type)

In [549]:
def duplicate_rows_by_col_key(df, df_collections):
    # Duplicate rows based on 'Col key'
    duplicated_rows = []
    for index, row in df.iterrows():
        if isinstance(row['Col key'], list):
            for key in row['Col key']:
                new_row = row.copy()
                collection_info = df_collections[df_collections['Key'] == key]
                if not collection_info.empty:
                    new_row['Collection_Key'] = key
                    new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                    new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                    duplicated_rows.append(new_row)
        else:
            key = row['Col key']
            new_row = row.copy()
            collection_info = df_collections[df_collections['Key'] == key]
            if not collection_info.empty:
                new_row['Collection_Key'] = key
                new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                duplicated_rows.append(new_row)

    # Create a new DataFrame with duplicated rows
    duplicated_df = pd.DataFrame(duplicated_rows)

    return duplicated_df

# Duplicating rows based on 'Col key' and collections information
duplicated_df = duplicate_rows_by_col_key(df, df_collections_2)

In [550]:
day_allitems = datetime.date.today().isoformat()

In [551]:
# df_fa = df['FirstName2']
# df_fa = pd.DataFrame(df_fa.tolist())
# df_fa = df_fa[0]
# df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
# df_new = pd.json_normalize(df_fa, errors='ignore') 
# df = pd.concat([df, df_new], axis=1)
# df['firstName'] = df['firstName'].fillna('null')
# df['lastName'] = df['lastName'].fillna('null')

In [552]:
df.to_csv('all_items.csv')
duplicated_df.to_csv('all_items_duplicated.csv')

In [553]:
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

# Countries 

In [554]:
# Read the csv file into a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')

# Dictionary to map non-proper country names to their proper names
country_map = {
    'british': 'UK',
    'great britain': 'UK',
    'UK' : 'UK', 
    'america' : 'United States',
    'United States of America' : 'United States',
    'Soviet Union': 'Russia', 
    'american' : 'United States',
    'United States' : 'United States',
    'russian' : 'Russia'
    # Add more mappings as needed
}

# Find the country names in the "title" column of the dataframe
found_countries = {}
for i, row in df.iterrows():
    title = str(row['Title']).lower()
    for country in pycountry.countries:
        name = country.name.lower()
        if name in title or (name + 's') in title:  # Check for singular and plural forms of country names
            proper_name = country.name
            found_countries[proper_name] = found_countries.get(proper_name, 0) + 1
    for non_proper, proper in country_map.items():
        if non_proper in title:
            found_countries[proper] = found_countries.get(proper, 0) + title.count(non_proper)

# Create a new dataframe containing the found countries and their counts
df_countries = pd.DataFrame({'Country': list(found_countries.keys()), 'Count': list(found_countries.values())})

In [555]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [556]:
df_countries=df_countries.sort_values(by='Count', ascending=False)
df_countries.reset_index(drop=True)

,Country,Count
0,UK,214
1,United States,178
2,Ukraine,96
3,Russia,67
4,Israel,48
...,...,...
74,North Macedonia,1
75,Bangladesh,1
76,Antarctica,1
77,Angola,1


In [557]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [558]:
df_countries.to_csv('countries.csv',index=False)

# NER analysis

In [559]:
import spacy
import nltk
import ast
from spacy.pipeline import EntityRecognizer

In [560]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MI6"}]
ruler.add_patterns(patterns)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
c:\Users\yaozk\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:865: UserWarning:

[W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible

In [561]:
def extract_entities(text):
    doc = nlp(text)
    orgs = []
    gpes = []
    people = []
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            orgs.append(entity.text)
        elif entity.label_ == 'GPE':
            gpes.append(entity.text)
        elif entity.label_ == 'PERSON':
            people.append(entity.text)
    return pd.Series({'ORG': orgs, 'GPE': gpes, 'PERSON': people})

In [562]:
df_title = df[['Title']].copy()
df_title = df_title.rename(columns={'Title':'Text'})
df_abstract = df[['Abstract']].copy()
df_abstract = df_abstract.rename(columns={'Abstract':'Text'})
df_one = pd.concat([df_title, df_abstract], ignore_index=True)
df_one['Text'] = df_one['Text'].fillna('')

In [563]:
df_one = pd.concat([df_one[['Text']], df_one['Text'].apply(extract_entities)], axis=1)
df_one = df_one.explode('GPE').reset_index(drop=True)
df_one = df_one.explode('ORG').reset_index(drop=True)
df_one = df_one.explode('PERSON').reset_index(drop=True)

In [564]:
df_one_g = df_one.copy()
df_one_g = df_one[['Text', 'GPE']]
# df_one_g = df_one_g.fillna('')
df_one_g = df_one_g.drop_duplicates(subset=['Text', 'GPE'])

gpe_counts = df_one_g['GPE'].value_counts().reset_index()
gpe_counts.columns = ['GPE', 'count']

# pd.options.display.max_rows = None


mapping_locations = {
    'the United States': 'USA',
    'The United States': 'USA',
    'US': 'USA',
    'U.S.': 'USA',
    'United States' : 'USA',
    'America' : 'USA',
    'the United States of America' : 'USA',
    'Britain' : 'UK',
    'the United Kingdom': 'UK',
    'U.K.' : 'UK',
    'Global Britain' : 'UK',
    'United Kingdom' : 'UK', 
    'the Soviet Union' : 'Russia',
    'The Soviet Union' : 'Russia',
    'USSR' : 'Russia',
    'Ukraine - Perspective' : 'Ukraine',
    'Ukrainian' : 'Ukraine',
    'Great Britain' : 'UK',
    'Ottoman Empire' : 'Turkey'
}
gpe_counts['GPE'] =gpe_counts['GPE'].replace(mapping_locations)
gpe_counts = gpe_counts.groupby('GPE').sum().reset_index()
gpe_counts.sort_values('count', ascending=False, inplace=True)
gpe_counts = gpe_counts.reset_index(drop=True)
gpe_counts.head(15)

,GPE,count
0,USA,406
1,UK,308
2,Russia,198
3,Ukraine,162
4,Germany,68
5,Israel,61
6,Canada,44
7,London,43
8,China,42
9,France,42


In [565]:
df_one_p = df_one.copy()
df_one_p = df_one[['Text', 'PERSON']]
# df_one_p = df_one_g.fillna('')
df_one_p = df_one_p.drop_duplicates(subset=['Text', 'PERSON'])

person_counts = df_one_p['PERSON'].value_counts().reset_index()
person_counts.columns = ['PERSON', 'count']

mapping_person = {
    'Putin' : 'Vladimir Putin',
    'Vladimir Putin' : 'Vladimir Putin',
    'Churchill' : 'Winston Churchill',
    'Hitler' : 'Adolf Hitler',
    'Biden' : 'Joe Biden',
    "John le Carré’s" : "John le Carré"
}

person_counts['PERSON'] =person_counts['PERSON'].replace(mapping_person)
person_counts = person_counts.groupby('PERSON').sum().reset_index()
person_counts.sort_values('count', ascending=False, inplace=True)

remove_person = ['MI6', 'Twitter', 'GRU'
          ]
person_counts = person_counts[~person_counts['PERSON'].isin(remove_person)]
person_counts = person_counts.reset_index(drop=True)

person_counts.head(15)

,PERSON,count
0,Vladimir Putin,52
1,Adolf Hitler,19
2,Winston Churchill,16
3,John le Carré,14
4,Joe Biden,11
5,Michael Morell,9
6,Loch K. Johnson,9
7,Stalin,7
8,David Omand,7
9,James Bond,6


In [566]:
df_one_o = df_one.copy()
df_one_o = df_one[['Text', 'ORG']]
# df_one_p = df_one_g.fillna('')
df_one_o = df_one_o.drop_duplicates(subset=['Text', 'ORG'])

org_counts = df_one_o['ORG'].value_counts().reset_index()
org_counts.columns = ['ORG', 'count']

mapping_organisations = {
    'The British Secret Intelligence Service' : 'SIS',
    'the British Secret Intelligence Service' : 'SIS',
    'The Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee - History' : 'Joint Intelligence Committee',
    'Central Intelligence Agency' : 'CIA',
    'the Central Intelligence Agency' : 'CIA',
    'the Foreign Office' : 'Foreign Office',
    'Schar School' : 'Schar School of Policy and Government',
    'the Secret Intelligence Service' : 'SIS',
    "George Mason University's" : "George Mason University",
    'JIC' : 'Joint Intelligence Committee'
}
org_counts['ORG'] =org_counts['ORG'].replace(mapping_organisations)
org_counts = org_counts.groupby('ORG').sum().reset_index()
org_counts.sort_values('count', ascending=False, inplace=True)

remove_orgs = ['Intelligence', 'Kremlin', 'Ultra', 'International Security', 'Intelligence Studies', 'Intelligence Analysis', 'OSINT']
org_counts = org_counts[~org_counts['ORG'].isin(remove_orgs)]
org_counts = org_counts.reset_index(drop=True)

org_counts.head(15)

,ORG,count
0,CIA,165
1,KGB,37
2,SIS,29
3,FBI,26
4,Allied,21
5,Foreign Office,21
6,Hamas,20
7,SOE,19
8,British Intelligence,18
9,NSA,17


In [567]:
gpe_counts.head(15).to_csv('gpe.csv')
person_counts.head(15).to_csv('person.csv')
org_counts.head(15).to_csv('org.csv')

In [568]:
polat = df[df['FirstName2']=='Polat Safi']

In [569]:
polat

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key
193,193,Mi̇lli̇ İsti̇hbarat Teşki̇latı (1826-2023) [Na...,Book,https://kronikkitap.com/kitap/milli-istihbarat...,Modern Türk istihbaratının bilinmeyenlerini ke...,https://www.zotero.org/groups/intelligence_bib...,2023-06-01,Polat Safi,Kronik Kitap,NaN,2023-06-21T20:24:30Z,"['CK5MNYPQ', 'AZ3BZ9BR']"
430,430,A First Button Done Up Wrongly: Revisiting Phi...,Journal article,https://doi.org/10.1080/23296151.2021.1983943,The Ottoman Special Organization made its way ...,https://www.zotero.org/groups/intelligence_bib...,2021-07-03,Polat Safi,NaN,Special Operations Journal,2023-01-31T21:52:57Z,['AZ3BZ9BR']
1710,1710,The Ottoman special organization-Teşkilat-ı Ma...,Thesis,http://repository.bilkent.edu.tr/handle/11693/...,The present level of the knowledge about the p...,https://www.zotero.org/groups/intelligence_bib...,2006,Polat Safi,NaN,NaN,2020-07-12T12:36:38Z,"['BNPYHVD4', '4BH8AYZE']"
1717,1717,The Ottoman Special organization - Teşkilat-ı ...,Thesis,http://repository.bilkent.edu.tr/handle/11693/...,"The usage or misuse of the terms, “intelligenc...",https://www.zotero.org/groups/intelligence_bib...,2012-09,Polat Safi,NaN,NaN,2020-06-05T10:17:16Z,"['BNPYHVD4', '4BH8AYZE']"
1718,1718,History in the Trench: The Ottoman Special Org...,Journal article,https://doi.org/10.1080/00263206.2011.553898,"Aside from a few exceptional works, studies on...",https://www.zotero.org/groups/intelligence_bib...,"January 1, 2012",Polat Safi,NaN,Middle Eastern Studies,2020-06-05T10:17:16Z,"['BNPYHVD4', '4BH8AYZE', 'AZ3BZ9BR']"
